# 99. Recover Binary Search Tree

Two elements of a binary search tree (BST) are swapped by mistake.

Recover the tree without changing its structure.

Note:
A solution using O(n) space is pretty straight forward. Could you devise a constant space solution?

## Solution 1. DFS Recursive Approach with Space Complexity of O(n)

Note: For DFS Recursive Approach, the stack for the recursive calls takes O(n) space.

In [ ]:
# Definition for a binary tree node.
# class TreeNode(object):
#     def __init__(self, x):
#         self.val = x
#         self.left = None
#         self.right = None

class Solution(object):
    def recoverTree(self, root):
        """
        :type root: TreeNode
        :rtype: void Do not return anything, modify root in-place instead.
        """
        
        lst = []
        err_lst = [None, None]
        
        def find_err(root):
            if root:
                find_err(root.left)
                #if lst != [] and root.val < lst[-1][1].val:
                
                if lst != []:
                    #print root.val, lst[-1][0]
                    if root.val < lst[-1][0]:
                        err_lst[0], err_lst[1] = root.val, root
                        
                lst.append([root.val, root])
                find_err(root.right)
        
        def swap(root):
            if root:
                if err_lst[0] == None: 
                    return 
                
                swap(root.left)
                
                if err_lst[0] != None and root.val > err_lst[0]:
                    #print root.val, err_lst[0]
                    root.val, err_lst[1].val = err_lst[0], root.val
                    err_lst[0], err_lst[1] = [None, None]
                    return
                
                swap(root.right)
        
        find_err(root)
        
        swap(root)

We can use generator to write more elegant codes using DFS recursive approach.

In [ ]:
# Definition for a binary tree node.
# class TreeNode(object):
#     def __init__(self, x):
#         self.val = x
#         self.left = None
#         self.right = None

class Solution(object):
    def recoverTree(self, root1):
        """
        :type root: TreeNode
        :rtype: void Do not return anything, modify root in-place instead.
        """
        
        def dfs(root):
            if root != None:
                for node in dfs(root.left):
                    yield node
                yield root
                for node in dfs(root.right):
                    yield node
            
        pre, first, second = None, None, None
        
        for node in dfs(root1):                     
            if pre and pre.val > node.val:
                    if first == None: 
                        first, second = pre, node
                    else:
                        second = node
                        break
            pre = node
            
        first.val, second.val = second.val, first.val 

## Solution 2. Iterative Approach

Some test cases:

1). 1 5 4 2
2). 1 5 3 4 2 6
3). 1 3 2 6
4). 2 1 
        

In [ ]:
# Definition for a binary tree node.
# class TreeNode(object):
#     def __init__(self, x):
#         self.val = x
#         self.left = None
#         self.right = None

class Solution(object):
    def recoverTree(self, root):
        """
        :type root: TreeNode
        :rtype: void Do not return anything, modify root in-place instead.
        """
        
        stack, lst = [], []
        swap = None
        node = root
        while node or stack:
            if node: 
                stack.append(node)
                node = node.left
            else:
                popped = stack.pop()
                node = popped.right
                if lst:
                    if popped.val < lst[-1].val: 
                        if not swap:
                            swap = [lst[-1], popped]
                        else:
                            swap[1] = popped
                            break
                lst.append(popped)
        
        swap[0].val, swap[1].val = swap[1].val, swap[0].val

In the above, we only checked the lst[-1]. We can replace lst with a variable named prev, which can save space and speed up the code a bit. Below is a slightly enhanced solution. 

In [ ]:
# Definition for a binary tree node.
# class TreeNode(object):
#     def __init__(self, x):
#         self.val = x
#         self.left = None
#         self.right = None

class Solution(object):
    def recoverTree(self, root):
        """
        :type root: TreeNode
        :rtype: void Do not return anything, modify root in-place instead.
        """
        
        stack, prev = [], None
        swap = None
        node = root
        while node or stack:
            if node: 
                stack.append(node)
                node = node.left
            else:
                popped = stack.pop()
                node = popped.right
                if prev:
                    if popped.val < prev.val: 
                        if not swap:
                            swap = [prev, popped]
                        else:
                            swap[1] = popped
                            break
                prev = popped
        
        swap[0].val, swap[1].val = swap[1].val, swap[0].val

## Solution 3. In-Order Morris Traversal (Time Complexity O(n), Space Complexity O(1))

In [ ]:
# Definition for a binary tree node.
# class TreeNode(object):
#     def __init__(self, x):
#         self.val = x
#         self.left = None
#         self.right = None

class Solution(object):
    def recoverTree(self, root):
        """
        :type root: TreeNode
        :rtype: void Do not return anything, modify root in-place instead.
        """

        def morris_in_order(root):
            current = root
            
            while current:
                if current.left == None:
                    yield current
                    current = current.right
                else:
                    pre = current.left
                    while pre != None and pre.right != None and pre.right != current:
                        pre = pre.right

                    if pre.right == None:
                        pre.right = current
                        current = current.left
                    else:
                        pre.right = None
                        yield current
                        current = current.right
        
        
        pre, first, second = None, None, None
        for node in morris_in_order(root):
            if pre:
                if pre.val > node.val:
                    if first == None:
                        first = pre
                        second = node
                    else: 
                        second = node
                        #NOTE: CANNOT BREAK HERE. This is because we need morris_in_order to revert
                        #the possible changes made previously
            pre = node
        
        first.val, second.val = second.val, first.val
